In [6]:
import numpy as np
import pandas as pd
#import tensorflow

In [7]:
#Load data
hand_log = pd.read_pickle("../fiveLogs.pickle")

In [39]:
hand_log.tail()

,Players,Steps,Cards,Result,hand_index
14994,"[ElDonatoro_2pn_2017, ASHE_2pn_2017]","[[0, 0, 0, c, 0, 1], [0, 1, 0, r, 50, 1], [0, ...","[[0, 0, 2, 3], [0, 0, 2, 0], [0, 1, 0, 1], [0,...","[300, -300]",14994
14995,"[ASHE_2pn_2017, ElDonatoro_2pn_2017]","[[0, 0, 0, c, 0, 1], [0, 1, 0, r, 50, 1], [0, ...","[[0, 0, 6, 2], [0, 0, 12, 1], [0, 1, 7, 2], [0...","[600, -600]",14995
14996,"[ElDonatoro_2pn_2017, ASHE_2pn_2017]","[[0, 0, 0, c, 0, 1], [0, 1, 0, r, 50, 1], [0, ...","[[0, 0, 0, 1], [0, 0, 4, 1], [0, 1, 7, 0], [0,...","[200, -200]",14996
14997,"[ASHE_2pn_2017, ElDonatoro_2pn_2017]","[[0, 0, 0, c, 0, 1], [0, 1, 0, r, 50, 1], [0, ...","[[0, 0, 7, 0], [0, 0, 6, 2], [0, 1, 1, 1], [0,...","[-100, 100]",14997
14998,"[ElDonatoro_2pn_2017, ASHE_2pn_2017]","[[0, 0, 0, c, 0, 1], [0, 1, 0, r, 50, 1], [0, ...","[[0, 0, 0, 1], [0, 0, 1, 3], [0, 1, 12, 0], [0...","[-100, 100]",14998


In [9]:
import constants

In [10]:
print(constants.cardRanks)
print(constants.suits)
print(dir(constants))

{'A': 13, 'K': 12, 'J': 10, 'Q': 11, 'T': 9, '3': 1, '2': 0, '5': 3, '4': 2, '7': 6, '6': 4, '9': 8, '8': 7}
{'h': 1, 's': 0, 'c': 3, 'd': 2}
['__builtins__', '__doc__', '__file__', '__name__', '__package__', 'big_blind', 'cardRanks', 'card_ranks', 'community_pool', 'hero', 'max_betting_rounds', 'max_players', 'max_raises', 'min_raise', 'num_players', 'path', 'small_blind', 'stack_sizes', 'suits']


In [11]:
hand_log.dtypes

Players    object
Steps      object
Cards      object
Result     object
dtype: object

In [12]:
hand_log["Steps"][0]
#action = [bet round, player_position, raising round, action, size_to, ante-flag]

[[0, 0, 0, 'c', 0, 1],
 [0, 1, 0, 'r', 50, 1],
 [0, 1, 1, 'r', 100, 1],
 [0, 1, 1, 'r', 200, 0],
 [0, 0, 2, 'c', 200, 0],
 [1, 0, 0, 'r', 400, 0],
 [1, 1, 0, 'c', 400, 0],
 [2, 0, 0, 'r', 1200, 0],
 [2, 1, 0, 'c', 1200, 0],
 [3, 0, 0, 'r', 20000, 0],
 [3, 1, 0, 'c', 20000, 0]]

In [13]:
hand_log["Cards"][0]
#[round, player, rank, suit]
#-1 player signifies community

[[0, 0, 4, 2],
 [0, 0, 0, 2],
 [0, 1, 1, 0],
 [0, 1, 2, 2],
 [1, -1, 12, 1],
 [1, -1, 10, 1],
 [1, -1, 9, 1],
 [2, -1, 8, 0],
 [3, -1, 4, 1]]

In [14]:
hand_log["Steps"].apply(len).mean()

8.598306553770252

In [15]:
hand_log["Cards"].apply(len).mean()

6.828521901460097

In [16]:
df_steps = pd.DataFrame(hand_log["Steps"][0], columns = ["bet round", "player_position",\
                                               "raising round", "action", "size_to", "ante-flag"])
df_steps

,bet round,player_position,raising round,action,size_to,ante-flag
0,0,0,0,c,0,1
1,0,1,0,r,50,1
2,0,1,1,r,100,1
3,0,1,1,r,200,0
4,0,0,2,c,200,0
5,1,0,0,r,400,0
6,1,1,0,c,400,0
7,2,0,0,r,1200,0
8,2,1,0,c,1200,0
9,3,0,0,r,20000,0


In [17]:
df_cards = pd.DataFrame(hand_log["Cards"][0], columns=["bet round", "player_position", "rank", "suit"])
df_cards

,bet round,player_position,rank,suit
0,0,0,4,2
1,0,0,0,2
2,0,1,1,0
3,0,1,2,2
4,1,-1,12,1
5,1,-1,10,1
6,1,-1,9,1
7,2,-1,8,0
8,3,-1,4,1


In [18]:
hand_log["hand_index"] = hand_log.index

In [19]:
num_steps = df_steps[df_steps["ante-flag"]==0]["ante-flag"].count()

p0_states = df_steps[(df_steps["player_position"] == 0) & (df_steps["ante-flag"] == 0)]
p1_states = df_steps[(df_steps["player_position"] == 1) & (df_steps["ante-flag"] == 0)]
#state:
#action x cards = [round, player, (raising round, action, size_to, ante-flag), (rank, suit)]

In [20]:
p0_states

,bet round,player_position,raising round,action,size_to,ante-flag
4,0,0,2,c,200,0
5,1,0,0,r,400,0
7,2,0,0,r,1200,0
9,3,0,0,r,20000,0


In [21]:
idx = 2
player_pos = 0
current_step = p0_states.iloc[idx].name
current_round = p0_states.iloc[idx]["bet round"]
known_actions = df_steps[df_steps.index < current_step]
known_cards = df_cards[((df_cards["player_position"] == -1) | (df_cards["player_position"] == player_pos)) \
                       & (df_cards["bet round"] <= current_round)]

print(known_actions)
print(known_cards)

   bet round  player_position  raising round action  size_to  ante-flag
0          0                0              0      c        0          1
1          0                1              0      r       50          1
2          0                1              1      r      100          1
3          0                1              1      r      200          0
4          0                0              2      c      200          0
5          1                0              0      r      400          0
6          1                1              0      c      400          0
   bet round  player_position  rank  suit
0          0                0     4     2
1          0                0     0     2
4          1               -1    12     1
5          1               -1    10     1
6          1               -1     9     1
7          2               -1     8     0


In [22]:
pd.concat([known_actions, known_cards], axis = 0)

,action,ante-flag,bet round,player_position,raising round,rank,size_to,suit
0,c,1.0,0,0,0.0,NaN,0.0,NaN
1,r,1.0,0,1,0.0,NaN,50.0,NaN
2,r,1.0,0,1,1.0,NaN,100.0,NaN
3,r,0.0,0,1,1.0,NaN,200.0,NaN
4,c,0.0,0,0,2.0,NaN,200.0,NaN
5,r,0.0,1,0,0.0,NaN,400.0,NaN
6,c,0.0,1,1,0.0,NaN,400.0,NaN
0,NaN,NaN,0,0,NaN,4.0,NaN,2.0
1,NaN,NaN,0,0,NaN,0.0,NaN,2.0
4,NaN,NaN,1,-1,NaN,12.0,NaN,1.0


In [23]:
#[row of bet round, player , [states]]
#a =[  [ "row 1" , ["state1","state2","state 3"]] , [ "row 2" , ["state 1_1", "state 1_2", "state 1_3"] ] ]

In [24]:
def create_state_array(stepList, cardList):
    
    df_steps = pd.DataFrame(stepList, columns = ["bet round", "player_position",\
                                               "raising round", "action", "size_to", "ante-flag"])
    df_cards = pd.DataFrame(cardList, columns=["bet round", "player_position", "rank", "suit"])
    num_steps = df_steps[df_steps["ante-flag"]==0]["ante-flag"].count()
    num_players = df_steps.player_position.max() + 1
    
    player_pos = 0
    reward_to = 0
    arr = []
    num_steps = len(df_steps)
    #can I make the states an array with dimensions: [bet round, player, [action: raise round, action, size_to, ante-flag],
    # and [cards: rank,suit]]?
    for step in range(3, num_steps):
        player_pos = df_steps.iloc[step, 1]
        bet_round = df_steps.iloc[step,0]
        known_actions = df_steps[df_steps.index < current_step]
        known_cards = df_cards[((df_cards["player_position"] == -1) | (df_cards["player_position"] == player_pos)) \
                       & (df_cards["bet round"] <= current_round)]
        done = step == num_steps - 1
        arr.append([player_pos, bet_round, [known_actions, known_cards], reward_to, df_steps.iloc[step], done]) #TODO: figure out reward

    return arr #[player, [(state1, reward_transitioning_to, action, done)]]
    

In [25]:
cardList = hand_log["Cards"][0]
stepList = hand_log["Steps"][0]

arr = create_state_array(stepList, cardList)

In [26]:
npArr = np.array(arr)

In [27]:
import pprint as pp
pp.pprint(npArr)

array([[1, 0,
        list([   bet round  player_position  raising round action  size_to  ante-flag
0          0                0              0      c        0          1
1          0                1              0      r       50          1
2          0                1              1      r      100          1
3          0                1              1      r      200          0
4          0                0              2      c      200          0
5          1                0              0      r      400          0
6          1                1              0      c      400          0,    bet round  player_position  rank  suit
2          0                1     1     0
3          0                1     2     2
4          1               -1    12     1
5          1               -1    10     1
6          1               -1     9     1
7          2               -1     8     0]),
        0,
        bet round            0
player_position      1
raising round        1
action    

In [28]:
np.array(npArr[0][2][0]) #bet state: bet round, player_position, raising round, action, size_to, ante-flag

array([[0L, 0L, 0L, 'c', 0L, 1L],
       [0L, 1L, 0L, 'r', 50L, 1L],
       [0L, 1L, 1L, 'r', 100L, 1L],
       [0L, 1L, 1L, 'r', 200L, 0L],
       [0L, 0L, 2L, 'c', 200L, 0L],
       [1L, 0L, 0L, 'r', 400L, 0L],
       [1L, 1L, 0L, 'c', 400L, 0L]], dtype=object)

In [29]:
np.array(npArr[0][2][1]) #card state: bet round  player_position  rank  suit

array([[ 0,  1,  1,  0],
       [ 0,  1,  2,  2],
       [ 1, -1, 12,  1],
       [ 1, -1, 10,  1],
       [ 1, -1,  9,  1],
       [ 2, -1,  8,  0]], dtype=int64)

In [30]:
npArr[0][3] #reward - NEED TO FIX

0

In [31]:
npArr[0][4] #action

bet round            0
player_position      1
raising round        1
action               r
size_to            200
ante-flag            0
Name: 3, dtype: object

In [32]:
npArr[0][5] #Done or not

False

In [33]:
#Ok, now how do we represent the bet and card state?
# We have: 
#- betting rounds (pre-flop, flop, turn, river = 0, 1 ,2 ,3)
#   - maximum raises per round (4 so [0, 1, 2 3])
#- players (0, 1, -1 for dealer) *until trying multiplayer
#   - cards
#      - action choice [r, c, f, collapse ante flag to this?]
#      - cost, which is negative of (reward - pot won if hand won)
#   


In [34]:
2**8 #binary representation of bets?

256

In [38]:
betRounds = 4
max_raises = 4
players = 3 #including community pool - or should I just make that a total pool for each player?
action = 4 #including ante flag
#cost is continuous - what to do about that?
cost = 8 #binary representation of cost - but could I collapse this to one with the action?
cards = (4, 13) # suit, rank
card_action_placeholders = 3
stacks = 8

state = np.zeros((betRounds, players, cards[0], cards[1], max_raises, cost, stacks)) #cards , (max_raises, cost/action)
state.shape

(4L, 3L, 4L, 13L, 4L, 8L, 8L)

In [37]:
print(reduce(np.multiply, state.shape))
print(np.sqrt(reduce(np.multiply, state.shape)))

19968
141.308173861


In [51]:
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# # define example
# data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
# values = np.array(data)
# print(values)
# # integer encode
# label_encoder = LabelEncoder()
# integer_encoded = label_encoder.fit_transform(values)
# print(integer_encoded)
# # binary encode
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)
# # invert first example
# inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
# print(inverted)

hello world
[7, 4, 11, 11, 14, 26, 22, 14, 17, 11, 3]
[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
h
